In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_supra',
    'd_po_header_spr',
    proc_date,
    'temp_d_po_header_spr'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_supra.d_po_header_spr
(
  po_code STRING,
  client_code STRING,
  wh_code STRING,
  wh_site_id STRING,
  external_code STRING,
  created_date DATE,
  est_receive_date DATE,
  contact_name STRING,
  contact_phone STRING,
  vendor_id STRING,
  vendor_name STRING,
  receiving_date DATE,
  finished_date DATE,
  canceled_date DATE,
  type STRING,
  condition_type STRING,
  source STRING,
  confirmed_date DATE,
  gr_date DATE,
  gr_status STRING,
  status STRING,
  calday DATE,
  proc_date DATE,
  reference_id STRING,
  file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
    delta.autoOptimize.optimizeWrite = true,
    delta.autoOptimize.autoCompact = true
)
""")

In [0]:
spark.sql(f"""DELETE FROM {catalog_name}.udp_wcm_silver_supra.d_po_header_spr 
WHERE proc_date IN (
  SELECT proc_date 
  FROM temp_d_po_header_spr
)""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_supra.d_po_header_spr
(
    po_code,
    client_code,
    wh_code,
    wh_site_id,
    external_code,
    created_date,
    est_receive_date,
    contact_name,
    contact_phone,
    vendor_id,
    vendor_name,
    receiving_date,
    finished_date,
    canceled_date,
    type,
    condition_type,
    source,
    confirmed_date,
    gr_date,
    gr_status,
    status,
    calday,
    proc_date,
    reference_id,
    file_creation_ts
)
SELECT
    po_code,
    client_code,
    wh_code,
    wh_site_id,
    external_code,
    try_cast(created_date AS DATE) AS created_date,         
    try_cast(est_receive_date AS DATE) AS est_receive_date, 
    contact_name,
    contact_phone,
    vendor_id,
    vendor_name,
    try_cast(receiving_date AS DATE) AS receiving_date,     
    try_cast(finished_date AS DATE) AS finished_date,       
    try_cast(canceled_date AS DATE) AS canceled_date,       
    type,
    condition_type,
    source,
    try_cast(confirmed_date AS DATE) AS confirmed_date,      
    try_cast(gr_date AS DATE) AS gr_date,                    
    gr_status,
    status,
    try_cast(calday AS DATE) AS calday,                      
    try_cast(proc_date AS DATE) AS proc_date,                
    reference_id,
    file_creation_ts
FROM temp_d_po_header_spr
WHERE proc_date IS NOT NULL
""")